In [8]:
import openai
import os
from dotenv import load_dotenv

# --- Configuration ---
# Replace with your actual OpenAI API key
# It's best to set this as an environment variable for security
# For example: openai.api_key = os.getenv("OPENAI_API_KEY")
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

client = openai.OpenAI()

explanation_text = None
try:
    print(f"\n🤖 Text wird generiert...")
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",  # Or your preferred model like "gpt-4o"
        messages=[
            {"role": "system", "content": "Du bist ein nützlicher und hilfsbereiter Assisstent, welcher klare und simple Erklärungen liefert."},
            {"role": "user", "content": f"Philosophiere über das Leben und die Reize des Meeres."}
        ],
        max_tokens=150,
        temperature=0.7
    )
    explanation_text = response.choices[0].message.content.strip()
    print(f"\n📜 Explanation:\n{explanation_text}")
except Exception as e:
    print(f"Error generating text with OpenAI: {e}")


🤖 Text wird generiert...

📜 Explanation:
Das Leben kann mit dem Meer verglichen werden - voller Tiefe, Geheimnisse und Schönheit. Das Meer, mit seiner endlosen Weite und seinen ständig wechselnden Gezeiten, symbolisiert die verschiedenen Phasen und Herausforderungen, die wir im Leben durchlaufen. Es kann sowohl ruhig und friedlich als auch stürmisch und wild sein, genauso wie unsere eigenen Emotionen und Erfahrungen.

Die Reize des Meeres sind vielfältig - die beruhigenden Klänge der Wellen, die erfrischende Brise, die salzige Luft und die faszinierende Vielfalt des Meeres


In [29]:
import playsound
from pathlib import Path

if explanation_text:
    print("\n🔊 Converting text to OpenAI speech...")
    cur_dir = os.getcwd()
    speech_file_path = cur_dir + "/speech.mp3"

    try:
        # Use .with_streaming_response for the API call
        with client.audio.speech.with_streaming_response.create(
            model="tts-1",
            voice="alloy",
            input=explanation_text,
            # response_format="mp3" # mp3 is default
        ) as response:
            # Now, use the .stream_to_file() method from the streaming response object
            response.stream_to_file(speech_file_path)

        print(f"Audio saved as {speech_file_path}")
        print("Playing audio...")
        playsound.playsound(str(speech_file_path))
        print("Playback finished.")

    except Exception as e:
        print(f"Error with OpenAI TTS or playing sound: {e}")
        if "401" in str(e): # Basic check for authentication error
             print("This might be an issue with your OpenAI API key or organization setup for TTS.")
        elif "Unsupported olha" in str(e) or "Unsupported model" in str(e): # Example for model/voice error
             print("Please ensure the selected voice and model for TTS are supported by your API access.")
else:
    print("No explanation text to convert to speech.")


🔊 Converting text to OpenAI speech...
Audio saved as /home/philipp/projects/gruene/podcast/speech.mp3
Playing audio...
Playback finished.


In [11]:
from google import genai
from google.genai import types
import wave

# Set up the wave file to save the output:
def wave_file(filename, pcm, channels=1, rate=24000, sample_width=2):
   with wave.open(filename, "wb") as wf:
      wf.setnchannels(channels)
      wf.setsampwidth(sample_width)
      wf.setframerate(rate)
      wf.writeframes(pcm)

GEMINI_API_KEY = os.getenv("GEMINI_API_TOKEN")
client = genai.Client(api_key=GEMINI_API_KEY)

response = client.models.generate_content(
   model="gemini-2.5-flash-preview-tts",
   contents="Lies bitte folgenden Text vor, als wärest du Teil eines Podcasts: Weißt du, manchmal stell ich mir vor, wie es wär, auf dem Meer zu leben. Weit draußen, nur Wasser bis zum Horizont, kein Straßenlärm, keine E-Mails, keine Termine. Nur Wind, Wellen – und das Hier und Jetzt.",
   config=types.GenerateContentConfig(
      response_modalities=["AUDIO"],
      speech_config=types.SpeechConfig(
         voice_config=types.VoiceConfig(
            prebuilt_voice_config=types.PrebuiltVoiceConfig(
               voice_name='Kore',
            )
         )
      ),
   )
)

data = response.candidates[0].content.parts[0].inline_data.data

file_name='out.wav'
wave_file(file_name, data) # Saves the file to current directory